In [1]:
# Import required modules
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.llms import Ollama
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import streamlit as st
import os
import time

In [2]:
FILEPATH    = "sample.pdf" 
LOCAL_MODEL = "llama2"
EMBEDDING   = "nomic-embed-text"

In [3]:
loader = PyPDFLoader(FILEPATH)
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [6]:
persist_directory = 'data'

vectorstore = Chroma.from_documents(
    documents=all_splits, 
    embedding=OllamaEmbeddings(model=EMBEDDING),
    persist_directory=persist_directory
)

# vectorstore.persist()

In [7]:
llm = Ollama(base_url="http://localhost:11434",
                                  model=LOCAL_MODEL,
                                  verbose=True,
                                  callback_manager=CallbackManager(
                                      [StreamingStdOutCallbackHandler()])
                                  )

retriever = vectorstore.as_retriever()

C:\Users\aanch\AppData\Local\Temp\ipykernel_1048\1380469864.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(base_url="http://localhost:11434",
C:\Users\aanch\AppData\Local\Temp\ipykernel_1048\1380469864.py:1: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(base_url="http://localhost:11434",


In [8]:
template = """ You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: {context}
    History: {history}

    User: {question}
    Chatbot:
    """
prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=template,
    )

memory = ConversationBufferMemory(
        memory_key="history",
        return_messages=True,
        input_key="question"
    )


C:\Users\aanch\AppData\Local\Temp\ipykernel_1048\2668035149.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [9]:
qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=retriever,
            verbose=True,
            chain_type_kwargs={
                "verbose": True,
                "prompt": prompt,
                "memory": memory,
            }
        )

In [10]:
query = "What clustering method have been implemented"
query += ". Only from this pdf. Keep it short"
# response = qa_chain(query)

In [12]:
qa_chain.invoke({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: 4.2
Hierarchical
Clustering:
Statistical
methods
such
as
hierarchical
clustering
analyze
data
by
building
clusters
in
a
hierarchy.
This
algorithm
employs
two
strategies,
agglomerative
and
divisive.
K
can
be
exactly
set
in
Hierarchical
Clustering
as
in
K-means,
and
n
is
the
number
of
data
points
such
that
n>K.
The
agglomerative
Hierarchical
Clustering
gathers
data
starting
with
n
clusters
and
continues
until
K
clusters
are
obtained.
The
divisive
begins
with
a
single
cluster
and
subsequently
splits
into
K
clusters
based
on
their
commonalities.
4.3
Affinity
Propagation
Clustering:
The
most
popular
clustering
technique
in
data
science
is
affinity
propagation.
Neither
the
cluster
size
nor
the
number
of
cluste

{'query': 'What clustering method have been implemented. Only from this pdf. Keep it short',
 'result': 'The clustering method implemented in the PDF is K-Means Clustering.'}